# Activity 2.2 - Transfer Learning

#### Objective(s):

This activity aims to introduce how to apply transfer learning

#### Intended Learning Outcomes (ILOs):
* Demonstrate how to build and train neural network
* Demonstrate how to apply transfer learning in neural network


#### Resources:
* Jupyter Notebook
* CIFAR-10 dataset

#### Procedures
Load the necessary libraries

In [1]:
from __future__ import print_function

import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Set the parameters

In [2]:
now = datetime.datetime.now
batch_size = 128
num_classes = 5
epochs = 5
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

Set how the input data is loaded

In [4]:
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

* Write a function to include all the training steps.
* Use the model, training set, test set and number of classes as function parameters


In [5]:
def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

Shuffle and split the data between train and test sets

In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


Create two datasets
* one with digits below 5
* one with 5 and above

In [7]:
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

* Define the feature layers that will used for transfer learning
* Freeze these layers during fine-tuning process

In [8]:
feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

Define the classification layers

In [9]:
classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

Create a model by combining the feature layers and classification layers

In [10]:
model = Sequential(feature_layers + classification_layers)

Check the model summary

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

 Train the  model on the digits 5,6,7,8,9

In [12]:
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 45s 189ms/step - loss: 1.6035 - accuracy: 0.2227 - val_loss: 1.5907 - val_accuracy: 0.2683
Epoch 2/5
230/230 [==============================] - 42s 182ms/step - loss: 1.5862 - accuracy: 0.2744 - val_loss: 1.5713 - val_accuracy: 0.4061
Epoch 3/5
230/230 [==============================] - 43s 188ms/step - loss: 1.5672 - accuracy: 0.3410 - val_loss: 1.5499 - val_accuracy: 0.5373
Epoch 4/5
230/230 [==============================] - 42s 184ms/step - loss: 1.5467 - accuracy: 0.4019 - val_loss: 1.5264 - val_accuracy: 0.6406
Epoch 5/5
230/230 [==============================] - 43s 187ms/step - loss: 1.5246 - accuracy: 0.4545 - val_loss: 1.5000 - val_accuracy: 0.7087
Training time: 0:03:35.157214
Test score: 1.5000303983688354
Test accuracy: 0.7087019085884094


Freeze only the feature layers

In [15]:
for l in feature_layers:
    l.trainable = False

Check again the summary and observe the parameters from the previous model

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

Train again the model using the 0 to 4 digits

In [17]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 19s 75ms/step - loss: 1.5890 - accuracy: 0.2751 - val_loss: 1.5676 - val_accuracy: 0.4304
Epoch 2/5
240/240 [==============================] - 16s 66ms/step - loss: 1.5585 - accuracy: 0.3521 - val_loss: 1.5335 - val_accuracy: 0.5552
Epoch 3/5
240/240 [==============================] - 16s 66ms/step - loss: 1.5268 - accuracy: 0.4268 - val_loss: 1.4992 - val_accuracy: 0.6307
Epoch 4/5
240/240 [==============================] - 16s 66ms/step - loss: 1.4959 - accuracy: 0.4873 - val_loss: 1.4645 - val_accuracy: 0.6830
Epoch 5/5
240/240 [==============================] - 15s 61ms/step - loss: 1.4633 - accuracy: 0.5509 - val_loss: 1.4301 - val_accuracy: 0.7282
Training time: 0:01:22.839931
Test score: 1.4300554990768433
Test accuracy: 0.7281572222709656


#### Supplementary Activity
Now write code to reverse this training process. That is, you will train on the digits 0-4, and then finetune only the last layers on the digits 5-9.

In [18]:
### type your code here
feature_layers2 = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

In [19]:
classification_layers2 = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

In [20]:
model2 = Sequential(feature_layers2 + classification_layers2)

In [21]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_4 (Activation)   (None, 26, 26, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_5 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 12, 12, 32)        0         
 g2D)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 12, 12, 32)        0         
                                                      

In [22]:
#training the model from 0-4
train_model(model2,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 45s 186ms/step - loss: 1.5880 - accuracy: 0.2525 - val_loss: 1.5557 - val_accuracy: 0.3754
Epoch 2/5
240/240 [==============================] - 46s 190ms/step - loss: 1.5422 - accuracy: 0.3651 - val_loss: 1.5043 - val_accuracy: 0.6120
Epoch 3/5
240/240 [==============================] - 45s 189ms/step - loss: 1.4958 - accuracy: 0.4665 - val_loss: 1.4494 - val_accuracy: 0.7595
Epoch 4/5
240/240 [==============================] - 45s 188ms/step - loss: 1.4412 - accuracy: 0.5583 - val_loss: 1.3863 - val_accuracy: 0.8258
Epoch 5/5
240/240 [==============================] - 44s 185ms/step - loss: 1.3804 - accuracy: 0.6287 - val_loss: 1.3147 - val_accuracy: 0.8603
Training time: 0:03:46.215785
Test score: 1.314663052558899
Test accuracy: 0.8602840900421143


In [27]:
#freezing the layers
for l in feature_layers2:
    l.trainable = False

In [25]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_4 (Activation)   (None, 26, 26, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_5 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 12, 12, 32)        0         
 g2D)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 12, 12, 32)        0         
                                                      

In [26]:
train_model(model2,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 16s 68ms/step - loss: 1.5700 - accuracy: 0.3200 - val_loss: 1.5284 - val_accuracy: 0.4454
Epoch 2/5
230/230 [==============================] - 15s 67ms/step - loss: 1.5287 - accuracy: 0.3579 - val_loss: 1.4832 - val_accuracy: 0.4917
Epoch 3/5
230/230 [==============================] - 15s 67ms/step - loss: 1.4863 - accuracy: 0.4092 - val_loss: 1.4395 - val_accuracy: 0.5633
Epoch 4/5
230/230 [==============================] - 16s 69ms/step - loss: 1.4474 - accuracy: 0.4620 - val_loss: 1.3974 - val_accuracy: 0.6264
Epoch 5/5
230/230 [==============================] - 16s 71ms/step - loss: 1.4089 - accuracy: 0.5143 - val_loss: 1.3565 - val_accuracy: 0.6715
Training time: 0:01:22.712103
Test score: 1.3564567565917969
Test accuracy: 0.6714667677879333


#### Conclusion

During this activity, I learned about transfer learning such as classification layers and how to freeze layers and what are its effects on the model. In the procedure when the layers are frozed the accuracy gets better. While in supplementary the accuracy got lower from 86.03% to 67.14%.
